<a href="https://colab.research.google.com/github/rocepeda/curso-estadistica-multivariante/blob/master/ciencia_datos_unicen_practica_2_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fundamentos de Ciencia de Datos - FCEx - UNICEN

# Unidad 2: Acceso a datos y análisis

## Clase Práctica 2: Acceso a datos y análisis (parte I)

En esta clase nos vamos a enfocar en ver:
Acceso a datos en Python. Lectura de archivos (CSV, JSON, MAT, XLSX). Pandas, Numpy. Pandas Summary

## 0 - Descargar datos con Python

Vamos a empezar la clase de hoy viendo cómo podemos hacer para descargar archivos directamente desde Python. Hacerlo desde acá es una muy buena práctica, porque nos permite generar códigos autocontenidos, que podamos ejecutar sin tener que andar moviendo nosotros los archivos de acá para allá.

Utilizaremos para esto una librería de Python llamada ```wget```, que nos permite traernos contenido de internet. El paquete no viene instalado por defecto en Colab, por lo que vamos a tener que instalarlo nosotrxs mismxs, usando el famoso ```pip install``` que vimos en clase.

Ahora bien, ¿cómo podemos hacer eso si estamos trabajando con bloques de código Python? Muy sencillo: vamos a usar el caracter de escape "!". Colab (y Jupyter) interpretan este caracter como una orden del estilo "todo lo que leas a partir de acá, lo tenés que ejecutar en la consola, no dentro de Python". Y como Colab usa Linux en el backend (como toda persona de bien), lo que siga al "!" va a tener que ser código [bash](https://www.freecodecamp.org/espanol/news/la-guia-definitiva-de-linea-de-comandos-de-linux-tutorial-completo-de-bash/), interpretable por la terminal de Unix. Por ejemplo, en este codiguito de acá abajo ejecutamos el comando ```ls``` de Unix, que sirve para ver el contenido de un directorio (algo así como el ```dir``` de la terminal de Windows).

In [ ]:
!ls

sample_data


Como por defecto estamos parados en la carpeta de archivos temporales de Google Colab, vamos a ver que dentro solamente tenemos una carpeta ```sample_data```, que adentro tiene algunos archivos para probar cosas vinculadas con data science, pero que no vamos a usar en clase.

Bueno, avancemos. Instalemos el paquete ```wget``` usando PIP:

In [ ]:
# instalamos el paquete para descargar archivos wget
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=2259dda9298e79ae9984cca27f4e333e752d0f6d51454b8394ed772476075174
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


Y una vez que lo tenemos, el siguiente paso es ya utilizar el paquete dentro de Python, y empezar a descargar archivos.

Dentro de ```wget``` tenemos la función ```download```, que, dada una URL que se pasa como string, descarga de internet su contenido. En este caso lo vamos a usar para descargar dos bases de datos que vamos a usar como ejemplo a lo largo de este Colab. Además, les vamos a mostrar cómo controlar si los archivos ya están descargados, usando otro paquete, ```os```, que contiene un montón de funcionalidades para trabajar con rutas y directorios, dentro del paquete ```path```:

In [ ]:
from wget import download
from os import path, listdir


# descargamos una base de datos en Excel
if not path.exists("itunes_data.xlsx"):
  download("https://github.com/PacktPublishing/Practical-Data-Science-with-Python/raw/main/4-Chapter-4/data/itunes_data.xlsx")
else:
  print("No vamos a bajar el archivo de itunes porque ya existe!")

# y una en CSV
if not path.exists("housing_pandas.csv"):
  download("https://ignaciorlando.github.io/datasets/housing_pandas.csv")
else:
  print("No vamos a bajar el archivo de casas porque ya existe!")

# chequeamos si los archivos están en el directorio donde estamos parados
lista_archivos = listdir("./")
print(f"El contenido de la carpeta es: {lista_archivos}")


El contenido de la carpeta es: ['.config', 'housing_pandas.csv', 'itunes_data.xlsx', 'sample_data']


Esta misma lógica que usamos para descargar un archivo XLSX y un CSV se puede usar para descargar cualquier tipo de archivo, solamente tenemos que estar atentos de estar pasando la URL correcta.

Otra cosa: tengan en cuenta que todos los archivos que están bajando se guardan en una carpeta temporal. Esto significa que cuando cierren Colab, se van a perder. Si los quisieran guardar en Google Drive, deberían montar su unidad de Google Drive desde el código. Tienen [instrucciones en este link](https://www.linkedin.com/pulse/google-colab-importando-archivos-cynthia-soto-cardinault/) si quieren hacerlo.

## 1 - Introducir datos manualmente

Va a ser muy raro que en esta materia les pidamos introducir bases de datos enteras de forma manual, porque se supone que trabajaremos con grandes bases de datos ya precargadas, almacenadas en archivos o en tablas en bases de datos. En algunos casos, sin embargo, está bueno ofrecerle al usuario la posibilidad de que meta algún valor por teclado, así que está bueno que vean cómo se hace en Python.

Para esto se usa una función ```input()```, que retorna siempre un string. Si lo quieren castear a un tipo específico, lo tienen que hacer uds a través del código.

Acá abajo les dejamos un ejemplo pavo de para qué podría servir:

In [ ]:
# Pedimos los datos al usuario
nombre = input("Decime tu nombre: ")
edad = int(input("¿Cuántos años tenés?: "))
promedio = float(input("¿Cuál es tu promedio en la facu?: "))

# Mostramos los datos ingresados
print("\nDatos del estudiante:")
print(f"Nombre: {nombre}")
print(f"Edad: {edad} años")
print(f"Promedio: {promedio}")

# Evaluamos el rendimiento
if promedio >= 8:
  print("¡Buen promedio! Seguí así.")
elif promedio >= 6:
  print("Vas bien, pero dale un poco más de rosca.")
else:
  print("Ojo al promedio, tenés que ponerle más ganas.")

Decime tu nombre: Nacho
¿Cuántos años tenés?: 35
¿Cuál es tu promedio en la facu?: 8.1

Datos del estudiante:
Nombre: Nacho
Edad: 35 años
Promedio: 8.1
¡Buen promedio! Seguí así.


## 3 - Practiquemos un análisis básico a partir de CSVs



### 3.1 - Lectura de archivos CSV

Los archivos CSV son archivos separados por coma. Son una forma muy útil de guardar tablas en formato texto, de una manera fácil de leer (si tenemos el software correcto, o si sabemos codear una buena forma de hacerlo).

Un archivo CSV luce más o menos como este ejemplo, donde se ven identificadores, nombres, apellidos, y edades de diferentes personas:

![](https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEg7YF4P9ba6qavKa9WBklMrvMSB2Euc2XzmThanA68n-sg1I8eQAVsAQBaSKapOpntihHFgoyLLMciM1Dg1mSShir6afHbhiZfi7bnQy-PzVPY_B3Yuk2w9QS6ZBnZh27BT1INh4vqj1IY/s1600/archivo+csv.PNG)

Para leer un archivo CSV hay un montón de opciones. Podemos hacerlo "a pedal", tratándolo como un archivo de texto convencional, leyendo las líneas una a una y separando por coma y salto de línea. Pero no tendría mucho sentido (a menos que lo quisiéramos hacer como un ejercicio de programación).

La mejor alternativa es que usemos [Pandas](https://pandas.pydata.org/), una de las librerías más útiles para data science, y que más vamos a usar a lo largo de todo el curso. Si sos curioso/a y ya tenés ganas de rosquear más allá de lo que vamos a aprender, te dejamos este libro para que lo chusmees durante la materia.

Bueno, pasemos a la acción. Vamos a trabajar con un dataset clásico para aprender análisis de datos, el [Boston House Price Dataset](https://www.kaggle.com/vikrishnan/boston-house-prices).

Los datos identifican suburbios o subregiones dentro de la ciudad de Boston, y datan del año 1970. Cada registro contiene una serie de campos que describen diferentes características del lugar, y estaba pensado para generar modelos de aprendizaje automático que permitan predecir el precio de las casas alojadas en ese distrito en particular.

A nosotros nos interesa en este caso hacer un análisis de esos datos, no realizar predicciones. Así que vamos a trabajar con ellos abriéndolos, primero, usando la función ```read_csv``` de Pandas:

In [ ]:
import pandas as pd

# abrimos el archivo usando una función específica de pandas
raw_dataset = pd.read_csv("housing_pandas.csv")

La función que acabamos de ejecutar permite levantar el archivo y asignarlo a una variable. La tabla que levantamos será un objeto de tipo ```DataFrame```.

Si queremos visualizarla, tendríamos que usar la función ```print()```, que convertirá el DataFrame a un string y lo mostrará por pantalla. Alternativamente, en Jupyter Notebooks podemos directamente escribir el nombre de la variable al final de un bloque de código. En Colab se incluyen además un montón de otras funciones útiles además de la visualización:

In [ ]:
raw_dataset

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


### 3.2 - Explorando por primera vez los datos



#### 3.2.1 - ¿Cuántos datos tenemos y de qué tipo son?

Siempre que nos hacemos esta pregunta buscamos responderla desde dos lugares:
- El número de registros que tenemos (o el tamaño de la muestra, para ser más precisos), y
- El número de variables (atributos, características, features) con el que contamos, para cada registro.

Cuando hablamos del tipo del dato, además, nos preguntamos qué formato tiene cada columna de la tabla: ¿cuáles son texto? ¿cuáles son numéricas? ¿de qué tipo?

Lo ideal en estos casos es usar el método ```info()``` de la clase DataFrame, que nos cuenta la cantidad de registros (```entries``` en idioma "pandómico"), el número de columnas, y que además cuenta la cantidad de valores nulos en cada columna, y el tipo de dato (```dtype```) asociado a cada una de las columnas:

In [ ]:
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


En este caso vemos que se trata de un archivo con 14 columnas y 506 registros, con 506 registros no nulos en todas las columnas (por ende, ninguna columna sin nulos), y todos en formato ```float64```. A priori esto puede indicarnos que todas las columnas son datos cuantitativos y contínuos... ¿pero lo serán? Lo vemos en un ratito.



Otro método interesante de la clase DataFrame es el ```head()```, que nos permite ver los primeros 5 registros de la tabla. Está bueno para explorar un poco cómo se ven los datos, y entender en líneas generales si lo levantamos bien o rompimos algo en el camino:

In [ ]:
raw_dataset.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 3.2.2 - ¿Qué pasa con cada variable del conjunto?

Normalmente siempre está bueno que nos tomemos un ratito para analizar las diferentes columnas del conjunto, y entender qué está pasando con cada una.

Idealmente, tendríamos que consultarle al stakeholder o al dueño de los datos a los que accedimos sobre la naturaleza de cada una de las columnas. En este caso, usemos la info disponible en la web de Kaggle, que les dejo traducida acá abajo:

    - CRIM     tasa de criminalidad per cápita por distrito
    - ZN       proporción de terrenos residenciales zonificados para lotes de más de 25,000 pies cuadrados.
    - INDUS    proporción de acres de negocios no minoristas por municipio.
    - CHAS     límite con el río Charles (= 1 si limita; 0 si no)
    - NOX      concentración de óxido nítrico (partes por 10 millones)
    - RM       cantidad promedio de habitaciones por vivienda
    - AGE      proporción de unidades ocupadas por sus propietarios construidas antes de 1940
    - DIS      distancias ponderadas a cinco centros de empleo de Boston
    - RAD      índice de accessibilidad a carreteras radiales
    - TAX      tasa de impuesto a la propiedad de valor total por U$D 10.000
    - PTRATIO  tasa alumno/profesor por localidad
    - B        1000(Bk - 0.63)^2, donde Bk es la proporción de habitantes de raza negra que viven en la zona
    - LSTAT    % de habitantes de clase social baja
    - MEDV     Valor medio de vivienda (en miles de dólares)

Algo muy útil es usar el método ```describe()``` de la clase ```DataFrame```, que extrae una serie de estadísticas descriptivas de todas las columnas:

- ```count```: el número de registros con esa columna.
- ```mean```: promedio de valores de esa columna.
- ```std```: desvío estándar.
- ```min```: valor mínimo de la columna.
- ```25%```: el cuartil 1 de la columna (ordenando los valores de menor a mayor, el que está ubicado al 25% de ese orden).
- ```50%```: el cuartil 2 de la columna (ordenando los valores de menor a mayor, el que está ubicado al 50% de ese orden - la mediana!)
- ```75%```: el cuartil 3 de la columna (ordenando los valores de menor a mayor, el que está ubicado al 75% de ese orden)
- ```max```: el valor máximo de la columna.

¿Lo usamos para mirar nuestros datos?

In [ ]:
raw_dataset.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


Si analizamos los resultados, podemos sacar algunas conclusiones interesantes:

- **CRIM** es una proporción, que no sabemos cómo está calculada, pero que tiene un valor máximo bastante alto (88.97). Algo raro para ser la cantidad de crímenes por cada habitante, ¿no? En el contexto de entender los precios de las diferentes subregiones, podríamos en principio asumir que este valor refleja el nivel de criminalidad de la zona, y que, a mayor criminalidad, probablemente sea menor el precio de comercialización.

- **ZN** indica la cantidad de terreno residencial de gran tamaño en una jurisdicción dentro de la ciudad, lo que puede afectar la densidad de población en esa región. Vemos que está expresada como una proporción en porcentaje, porque el mínimo es 0 y el máximo es 100. Vemos también que hay bastantes zonas con poca proporción residencial de gran tamaño.

- **INDUS** también es una proporción, que mide el espacio de la región dedicado a emprendimientos comerciales que no son almacenes o negocios de venta directa minorista. De nuevo, vemos que no es un número entre 0 y 1, con lo cual pareciera estar expresado ya en porcentajes.

- **CHAS** indica si un vecindario está cerca del río Charles ([el río que corta al medio a Boston](https://www.google.com/maps/place/Charles+River+Esplanade/@42.3536851,-71.0868563,15.62z/data=!4m6!3m5!1s0x89e37a0a0667d9bf:0xf01e15e2d5decf7a!8m2!3d42.3555785!4d-71.0788785!16s%2Fm%2F0j441wr?entry=ttu)), y es claramente una variable booleana. Usarla de esta forma numérica no está mal, pero puede dar lugar a confusión, deberíamos trabajarla como una variable booleana.

- **NOX** es una concentración de un material contaminante, que en este caso está medida en partes por millón. No apreciera haber problemas con esta variable.

- **RM** es el número promedio de habitaciones por vivienda. Por habitaciones en este caso nos referimos a ambientes, no a cuartos donde dormir, por ende tiene sentido que veamos un mínimo en torno a 3.5 y un máximo de 8.7. Vemos decimales, pero tienen que ver con que estamos mirando promedio para todas las casas de esa subregión de la ciudad.

- **AGE** es la proporción de edificios que datan de 1940, dentro de esa región que están todavía ocupados por sus dueños. Nuevamente, es una proporción, que debería estar entre 0 y 1, pero al ver que el máximo es 100, nos da la pauta de que está expresada como porcentaje. Podría usarse como una suerte de medida de antigüedad del barrio, que puede influir o bien en las necesidades de mantenimiento de las casas, o bien también en la estética general del barrio.

- **DIS** son las distancias ponderadas a los cinco centros de empleo de Boston. No sabemos en qué unidad de medida estará, pero probablemente sean millas. Nos está dando una pauta sobre qué tan cerca estamos de lugares a los que ir a trabajar.

- **RAD** es un índice de accesibilidad a las autopistas radiales de la ciudad. No sabemos cómo se calcula, porque vemos que el máximo es 24 y el mínimo es 1 (tal vez indique la cantidad de autopistas a las que puede acercarse, pero no lo sabemos). Puede entenderse como un factor importante para la movilidad y la conectividad.

- **TAX** es la tasa de impuesto a la propiedad de valor total por cada U$D 10.000. Nos sirve para calcular cuánto paga de impuestos cada vivienda, según su cotización fiscal. Esto puede tener un cierto vínculo con el costo de adquirir una vivienda (si tengo que pagar muchos impuestos, probablemente sea cara).

- **PTRATIO** es la proporción de alumnos por cada maestro de esa zona. refleja la calidad del sistema educativo local, un favor importante para las familias que buscan comprar viviendas.

- **B** es quizá el dato más polémico de todos (recordemos que estamos trabajando con un conjunto de datos de los años '70). Es un indicador numérico que nos refleja la proporción de personas afroamericanas dentro de la zona, algo así como un indicador de la composición racial de la región. Podríamos usarla como una variable para estudiar potenciales sesgos raciales.

- **LSTAT** refleja el porcentaje de la población de menor estatus socioeconómico. Nuevamente, está expresado como porcentaje. Podría ayudarnos a entender la dinámica social y económica de un área.

- **MEDV** es el valor medio de las viviendas ocupadas por sus duseños, en miles de dólares. Representa el valor típico de las propiedades en ese vecindario, y se puede utiliar como indicador principal en el analisis del mercado inmobiliario.


#### 3.2.3 - Algunas transformaciones básicas

Veamos algunas transformaciones muy básicas por ahora que podemos hacer sobre los datos. Nos vamos a enfocar solamente en operaciones matemáticas sencillas, y en cambios de tipos.

Si quisiéramos por ejemplo cambiar el tipo de la columna ```CHAS``` por una variable booleana, podemos hacer:

In [ ]:
# cambiamos el tipo a booleano
raw_dataset["CHAS"] = raw_dataset["CHAS"].astype(bool)
# mostramos cómo queda el dataset
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    bool   
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: bool(1), float64(13)
memory usage: 52.0 KB


Y si quisiéramos, podríamos dividir todas las proporciones que sabemos que son porcentajes por 100, así no nos quedan expresadas más como porcentajes:

In [ ]:
# dividimos las columnas que son porcentajes por 100
raw_dataset["ZN"] = raw_dataset["ZN"] / 100
raw_dataset["INDUS"] = raw_dataset["INDUS"] / 100
raw_dataset["AGE"] = raw_dataset["AGE"] / 100
raw_dataset["LSTAT"] = raw_dataset["LSTAT"] / 100

# mostramos cómo queda el dataset
raw_dataset.describe()


,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,0.113636,0.111368,0.554695,6.284634,0.685749,3.795043,9.549407,408.237154,18.455534,356.674032,0.126531,22.532806
std,8.601545,0.233225,0.068604,0.115878,0.702617,0.281489,2.105710,8.707259,168.537116,2.164946,91.294864,0.071411,9.197104
min,0.006320,0.000000,0.004600,0.385000,3.561000,0.029000,1.129600,1.000000,187.000000,12.600000,0.320000,0.017300,5.000000
25%,0.082045,0.000000,0.051900,0.449000,5.885500,0.450250,2.100175,4.000000,279.000000,17.400000,375.377500,0.069500,17.025000
50%,0.256510,0.000000,0.096900,0.538000,6.208500,0.775000,3.207450,5.000000,330.000000,19.050000,391.440000,0.113600,21.200000
75%,3.677083,0.125000,0.181000,0.624000,6.623500,0.940750,5.188425,24.000000,666.000000,20.200000,396.225000,0.169550,25.000000
max,88.976200,1.000000,0.277400,0.871000,8.780000,1.000000,12.126500,24.000000,711.000000,22.000000,396.900000,0.379700,50.000000


#### 3.2.3 - Análisis exploratorio

Podemos usar algunas de las herramientas de Pandas para hacer algo de análisis exploratorio de los datos.

Por ejemplo, los DataFrame de Pandas proveen la función ```groupby```, que permite agrupar los datos por una categoría. Podríamos usarlo para agrupar según la variable ```CHAS```, y ver qué ocurre con la media de precios para viviendas cercanas al río versus las que no lo están:

In [ ]:
# Agrupamos los datos por la variable CHAS
grouped_data = raw_dataset.groupby('CHAS')
# Calculamos la media de MEDV para cada grupo
mean_medv_by_chas = grouped_data['MEDV'].mean()
# Mostramos los resultados
print(mean_medv_by_chas)
# Calculamos la mediana de MEDV para cada grupo
median_medv_by_chas = grouped_data['MEDV'].median()
# Mostramos los resultados
print(median_medv_by_chas)


CHAS
False    22.093843
True     28.440000
Name: MEDV, dtype: float64
CHAS
False    20.9
True     23.3
Name: MEDV, dtype: float64



También podríamos por ejemplo contar cuántas regiones de la muestra efectivamente lindan con el río, y cuántas no. Para eso, vamos a usar las operaciones indexadas de Pandas, y usaremos el atributo ```shape```, que nos devuelve el tamaño de la tabla:

In [ ]:
# contamos la cantidad de regiones limítrofes al río
num_river = raw_dataset[raw_dataset["CHAS"]==True].shape[0]

print(f"Cantidad de regiones que lindan con el rio = {num_river}")

Cantidad de regiones que lindan con el rio = 35


Podemos también calcular cuántas regiones tienen valor de ```AGE``` = 100%. Simplemente necesitamos primero obtener el valor máximo de la columna, y luego filtrar y contar el número de registros para ese valor máximo particular:

In [ ]:
# calculamos el máximo valor de AGE
max_age = raw_dataset["AGE"].max()

# filtramos el DataFrame para obtener los registros con AGE igual al máximo
registros_max_age = raw_dataset[raw_dataset["AGE"] == max_age]

# contamos el número de registros
num_registros = registros_max_age.shape[0]

# imprimimos el resultado
print(f"Hay {num_registros} registros con AGE igual al máximo valor, que es {max_age}")


Hay 43 registros con AGE igual al máximo valor, que es 1.0


Otra herramienta útil que tenemos es el ```value_counts()```, que nos cuenta la cantidad de registros repetidos que tenemos en un DataFrame. Si lo usamos inteligentemente, podemos por ejemplo contar cuántos valores repetidos tenemos dentro de una cierta columna:

In [ ]:
# Contamos la cantidad de registros según cada valor único de RAD
conteo_por_rad = raw_dataset["RAD"].value_counts()

# Mostramos los resultados
print(conteo_por_rad)


RAD
24.0    132
5.0     115
4.0     110
3.0      38
6.0      26
2.0      24
8.0      24
1.0      20
7.0      17
Name: count, dtype: int64


También tenemos funciones que nos permiten ordenar ascendente o descendentemente nuestros resultados. Podemos por ejemplo usarlo para ordenar el índice (en este caso, el valor único de RAD), usando ```sort_index()```:

In [ ]:
# Ordenamos el conteo por el valor de RAD de forma descendente
conteo_por_rad_ordenado = conteo_por_rad.sort_index(ascending=False)

# Mostramos los resultados ordenados
print(conteo_por_rad_ordenado)

RAD
24.0    132
8.0      24
7.0      17
6.0      26
5.0     115
4.0     110
3.0      38
2.0      24
1.0      20
Name: count, dtype: int64


O bien usar ```sort_values()```, que nos va a ordenar según los valores:

In [ ]:
# Ordenamos el conteo de forma descendente
conteo_por_rad_ordenado = conteo_por_rad.sort_values(ascending=False)

# Mostramos los resultados ordenados
print(conteo_por_rad_ordenado)

RAD
24.0    132
5.0     115
4.0     110
3.0      38
6.0      26
2.0      24
8.0      24
1.0      20
7.0      17
Name: count, dtype: int64


A lo largo del curso vamos a ver montones de otras funcionalidades de Pandas, súper útiles para hacer este tipo de análisis. Pero no sean manija!!  Aguanten!!

# 4 - Archivos pickle: conservemos lo que hicimos!

Los archivos pickle son muy útiles para guardar los resultados de un proceso como este, y poder seguir operando sobre ellos más adelante. No es un tipo de dato recomendado por ejemplo para comunicar resultados, ya que necesitaríamos que la persona que lo use sepa Python para poder abrirlo.

Vamos a ver un ejemplo. Anteriormente hicimos algunos cambios de tipos sobre nuestro DataFrame. Si quisiéramos conservarlo para hacer experimentos en otra notebook más adelante, podríamos salvar el DataFrame directamente como un archivo Pickle.

Pandas nos ofrece un método muy cómodo para esto, el ```to_pickle()```:

In [ ]:
# guardamos el dataframe en un archivo pickle
raw_dataset.to_pickle("housing_pandas.pkl")

También tenemos otros métodos igual de útiles para otros formatos de archivo, como el ```to_csv()``` o el ```to_excel()```, que tienen un poco más de sentido para compartir resultados:

In [ ]:
# guardamos el dataframe en un archivo csv
raw_dataset.to_csv("housing_pandas.csv")

# guardamos el dataframe en un archivo excel
raw_dataset.to_excel("housing_pandas.xlsx")

Pero no nos vayamos por las ramas, volvamos al pickle. Si quisiéramos podríamos usar este formato de archivo para guardar más de un objeto al mismo tiempo. Podríamos hacerlo usando la librería ```pickle``` de Python, que ya viene instalada por defecto, la sintaxis ```with```, y el método ```dump```, que "arroja" ("dump") los objetos dentro del archivo. Lo importante es que vamos a tener que crear primero un diccionario con lo que queremos guardar, y luego guardarlo:

In [ ]:
import pickle

# Guardamos ambos objetos en un mismo archivo pickle
with open("dos_objetos.pkl", "wb") as f:
  data = {"raw_dataset": raw_dataset, "conteo_por_rad_ordenado": conteo_por_rad_ordenado}
  pickle.dump(data, f)

Gracias al ```with``` no necesitamos tener que acordarnos si cerramos el archivo, porque en cuanto se termina el ```with```, el archivo se cierra.

Si quiero leer los datos, tengo que repetir el ```with```, pero usando ```load``` en vez de ```dump```, y aclarando que estamos leyendo (pasamos ```rb``` en lugar de ```wb```):

In [ ]:
with open("dos_objetos.pkl", "rb") as f:
 data = pickle.load(f)

print(type(data))
for key in data.keys():
  print(key)
  print(data[key])

<class 'dict'>
raw_dataset
        CRIM    ZN   INDUS   CHAS    NOX     RM    AGE     DIS  RAD    TAX  \
0    0.00632  0.18  0.0231  False  0.538  6.575  0.652  4.0900  1.0  296.0   
1    0.02731  0.00  0.0707  False  0.469  6.421  0.789  4.9671  2.0  242.0   
2    0.02729  0.00  0.0707  False  0.469  7.185  0.611  4.9671  2.0  242.0   
3    0.03237  0.00  0.0218  False  0.458  6.998  0.458  6.0622  3.0  222.0   
4    0.06905  0.00  0.0218  False  0.458  7.147  0.542  6.0622  3.0  222.0   
..       ...   ...     ...    ...    ...    ...    ...     ...  ...    ...   
501  0.06263  0.00  0.1193  False  0.573  6.593  0.691  2.4786  1.0  273.0   
502  0.04527  0.00  0.1193  False  0.573  6.120  0.767  2.2875  1.0  273.0   
503  0.06076  0.00  0.1193  False  0.573  6.976  0.910  2.1675  1.0  273.0   
504  0.10959  0.00  0.1193  False  0.573  6.794  0.893  2.3889  1.0  273.0   
505  0.04741  0.00  0.1193  False  0.573  6.030  0.808  2.5050  1.0  273.0   

     PTRATIO       B   LSTAT  MEDV  

Y eso es todo por hoy! Aprendimos de todo: cómo meter valores por teclado, cómo descargar archivos con Python, cómo leer archivos CSV usando Pandas, cómo hacer un análisis exploratorio inicial y algunas transformaciones, y finalmente cómo exportar nuestros resultados en archivos CSV, de Excel y pickle.

Con todo esto que vieron, ya pueden hacer el práctico 2!